# 영화 데이터 수집 프로젝트 

# 0.라이브러리 불러오기

In [ ]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os
from IPython.display import display

## 1. 영화 사이트 요청 및 파싱

In [ ]:
movie_ranking = requests.get("https://www.moviechart.co.kr/rank/realtime/index/image")
if movie_ranking.status_code == 200:
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')

## 2. 저장 디렉토리 준비, 파일명 규칙 생성

In [ ]:
image_dir = 'images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
pattern = r'[\\/:"*?<>|]'

## 3. 영화 제목, 이미지 태그 추출

In [ ]:
title_list = soup.select(".movieBox-list .movie-title a")
image_list = soup.select(".movieBox-list .movieBox-item img")

## 4. 이미지 수집 및 저장 프로세스

In [ ]:
for title, image in zip(title_list, image_list):
    image_src = image.get('src')
    image_response = requests.get("https://www.moviechart.co.kr" + image_src)
    img = Image.open(BytesIO(image_response.content))
    image_filename = re.sub(pattern, '', title.text)
    img.save(os.path.join(image_dir, image_filename + ".png"))

## 5. 영화 랭킹, 포스터 자동 수집

In [ ]:
print(f"수집한 영화 수: {len(title_list)}")

In [ ]:
base_url = "https://www.moviechart.co.kr"
ranking_url = base_url + "/rank/realtime/index/image"
image_dir = "images"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# 1. 실시간 사이트에서 HTML 가져오기
resp = requests.get(ranking_url)
soup = BeautifulSoup(resp.content, 'html.parser')

# 2. 영화 제목과 이미지 경로 리스트 파싱
title_tags = soup.select(".movieBox-list .movie-title a")
img_tags = soup.select(".movieBox-list .movieBox-item img")

movies = []
for rank, (title_tag, img_tag) in enumerate(zip(title_tags, img_tags), 1):
    title = title_tag.text.strip()
    img_src = img_tag.get('src')
    if not img_src.startswith("http"):
        img_url = base_url + img_src
    else:
        img_url = img_src
    # 파일명 정제
    import re
    pattern = r'[\\/:"*?<>|]'
    filename = re.sub(pattern, '', title) + ".png"
    filepath = os.path.join(image_dir, filename)
    # 필요시 이미지 새로 저장
    if not os.path.exists(filepath):
        img_resp = requests.get(img_url)
        img = Image.open(BytesIO(img_resp.content))
        img.save(filepath)
    # 리스트에 저장
    movies.append({'rank': rank, 'title': title, 'filepath': filepath})

# 3. 랭킹, 제목, 이미지 동시 출력
for movie in movies:
    print(f"랭킹: {movie['rank']}")
    print(f"제목: {movie['title']}")
    img = Image.open(movie['filepath'])
    display(img)
    print('-' * 30)

# 6. 결론

실시간 영화 랭킹과 포스터 이미지를 자동 수집, 저장하고,<br>
최신 정보를 효과적으로 시각화하는 파이프라인 구현<br>
신뢰성있는  멀티미디어데이터 확보하였다고 판단<br>
멀티미디어데이터 수집 프로젝트를 달성했다고 판단.